In [18]:
import pandas as pd
import re

In [10]:
racism_data = pd.read_csv('twitter_racism_parsed_dataset.csv')

In [11]:
racism_data.head()

,index,id,Text,Annotation,oh_label
0,5.767493e+17,5.767493e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.408905e+17,5.408905e+17,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,none,0
2,5.678433e+17,5.678433e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.766462e+17,5.766462e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.713492e+17,5.713492e+17,@carolinesinders @herecomesfran *hugs*,none,0


In [12]:
racism_data_1 = racism_data[racism_data['oh_label'] == 1]  # Records where oh_label is 1
racism_data_0 = racism_data[racism_data['oh_label'] == 0]  # Records where oh_label is 0

In [13]:
num_duplicates = racism_data_1.duplicated(subset=['index', 'id']).sum()
print(f"Total duplicate rows based on index and id: {num_duplicates}")

Total duplicate rows based on index and id: 0


In [14]:
def format_id(value):
    value_int = int(value)  # Convert scientific notation to integer
    value_str = str(value_int)  # Convert to string for formatting
    return f"{value_str[:4]}-{value_str[4:8]}-{value_str[8:12]}-{value_str[12:16]}-{value_str[16:]}"  # Add hyphens

In [15]:
# Apply the transformation to 'index' and 'id'
racism_data_1['formatted_index'] = racism_data_1['index'].apply(format_id)
racism_data_1['formatted_id'] = racism_data_1['id'].apply(format_id)

C:\Users\Jeevan\AppData\Local\Temp\ipykernel_13320\2524733444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  racism_data_1['formatted_index'] = racism_data_1['index'].apply(format_id)
C:\Users\Jeevan\AppData\Local\Temp\ipykernel_13320\2524733444.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  racism_data_1['formatted_id'] = racism_data_1['id'].apply(format_id)


In [17]:
# Display the updated dataframe
racism_data_1[['formatted_index', 'formatted_id']].head()

,formatted_index,formatted_id
0,5767-4933-6190-5250-56,5767-4933-6190-5250-56
2,5678-4332-0338-1249-92,5678-4332-0338-1249-92
8,5646-0409-0752-0609-92,5646-0409-0752-0609-92
9,5633-6995-6734-0300-16,5633-6995-6734-0300-16
10,5669-8397-0571-5790-08,5669-8397-0571-5790-08


In [31]:
def extract_and_remove_mentions_hashtags_links(text):
    usernames = re.findall(r'@\w+', text)  # Extract usernames
    hashtags = re.findall(r'#\w+', text)  # Extract hashtags
    links = re.findall(r'http[s]?://\S+', text)  # Extract links starting with http
    clean_text = re.sub(r'@\w+|#\w+|http[s]?://\S+', '', text).strip()  # Remove usernames, hashtags, and links
    return clean_text, ' '.join(usernames), ' '.join(hashtags), ' '.join(links)  # Return cleaned text and extracted words

In [32]:
racism_data_1[['cleaned_text', 'users', 'hashtags','links']] = racism_data_1['Text'].apply(lambda x: pd.Series(extract_and_remove_mentions_hashtags_links(x)))

C:\Users\Jeevan\AppData\Local\Temp\ipykernel_13320\2912312543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  racism_data_1[['cleaned_text', 'users', 'hashtags','links']] = racism_data_1['Text'].apply(lambda x: pd.Series(extract_and_remove_mentions_hashtags_links(x)))
C:\Users\Jeevan\AppData\Local\Temp\ipykernel_13320\2912312543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  racism_data_1[['cleaned_text', 'users', 'hashtags','links']] = racism_data_1['Text'].apply(lambda x: pd.Series(extract_and_re

In [33]:
racism_data_1[['cleaned_text', 'users', 'hashtags','links']].head()

,cleaned_text,users,hashtags,links
0,Muslim mob violence against Hindus in Banglade...,@AAlwuhaib1977,#Islam,http://t.co/C1JBWJwuRc
2,Islamophobia is like the idea of Naziphobia. I...,@jncatron @isra_jourisra @AMPalestine,,
8,Islam invaded and conquered 2/3 of Christiando...,@HuffPostRelig,,
9,Do you approve of your pedophile prophet rapin...,@semzyxx,,
10,Problem with vile Muslims is that they try to ...,@watan71969 @geeky_zekey,,


In [35]:
# Reorder and drop unnecessary columns
racism_data_1 = racism_data_1[['formatted_index', 'formatted_id', 'cleaned_text', 'Annotation', 'oh_label', 'users', 'hashtags', 'links']]

In [36]:
# Save the cleaned dataframe to a CSV file
racism_data_1.to_csv('racism_cleaned_dataset.csv', index=False)